<a href="https://colab.research.google.com/github/gabrielhidd/BNDES/blob/main/bndes_informes_trimestrais_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd 

#1.0 - Coleta e limpeza dos dados da CVM

In [2]:
deletar = ['QT_COTA', 'VL_PATRIM_COTA', 'NR_COTST', 'ENTID_INVEST', 'VL_CAP_COMPROM', 'VL_INVEST_FIP_COTA', 'NR_COTST_SUBSCR_PF', 'PR_COTA_SUBSCR_PF', 'NR_COTST_SUBSCR_PJ_NAO_FINANC', 'PR_COTA_SUBSCR_PJ_NAO_FINANC', 
           'NR_COTST_SUBSCR_BANCO', 'PR_COTA_SUBSCR_BANCO', 'NR_COTST_SUBSCR_CORRETORA_DISTRIB', 'PR_COTA_SUBSCR_CORRETORA_DISTRIB', 'NR_COTST_SUBSCR_PJ_FINANC', 'PR_COTA_SUBSCR_PJ_FINANC', 'NR_COTST_SUBSCR_INVNR', 
           'PR_COTA_SUBSCR_INVNR', 'NR_COTST_SUBSCR_EAPC', 'PR_COTA_SUBSCR_EAPC', 'NR_COTST_SUBSCR_EFPC', 'PR_COTA_SUBSCR_EFPC', 'NR_COTST_SUBSCR_RPPS', 'PR_COTA_SUBSCR_RPPS', 'NR_COTST_SUBSCR_SEGUR', 'PR_COTA_SUBSCR_SEGUR', 
           'NR_COTST_SUBSCR_CAPITALIZ', 'PR_COTA_SUBSCR_CAPITALIZ', 'NR_COTST_SUBSCR_FII', 'PR_COTA_SUBSCR_FII', 'NR_COTST_SUBSCR_FI', 'PR_COTA_SUBSCR_FI', 'NR_COTST_SUBSCR_DISTRIB', 'PR_COTA_SUBSCR_DISTRIB', 
           'NR_COTST_SUBSCR_OUTRO', 'PR_COTA_SUBSCR_OUTRO', 'CLASSE_COTA', 'NR_COTST_SUBSCR_CLASSE', 'QT_COTA_SUBSCR_CLASSE', 'QT_COTA_INTEGR_CLASSE', 'VL_QUOTA_CLASSE', 'DIREITO_POLIT_CLASSE', 'DIREITO_ECON_CLASSE', 
           'QT_COTA_INTEGR', 'VL_CAP_INTEGR']

##1.1 - CVM 2021

In [3]:
cvm_2021 = pd.read_csv('http://dados.cvm.gov.br/dados/FIP/DOC/INF_TRIMESTRAL/DADOS/inf_trimestral_fip_2021.csv', sep = ";", encoding = 'ISO-8859-1')

In [4]:
cvm_2021.drop(columns = deletar, axis = 1, inplace = True)

In [5]:
cvm_2021['DT_COMPTC'] = pd.to_datetime(cvm_2021['DT_COMPTC'], yearfirst= True)

##1.2 - CVM 2020

In [6]:
cvm_2020 = pd.read_csv('http://dados.cvm.gov.br/dados/FIP/DOC/INF_TRIMESTRAL/DADOS/inf_trimestral_fip_2020.csv', sep = ";", encoding = 'ISO-8859-1')

In [7]:
cvm_2020.drop(columns = deletar, axis = 1, inplace = True)

In [8]:
cvm_2020['DT_COMPTC'] = pd.to_datetime(cvm_2020['DT_COMPTC'], yearfirst= True)

##2.0 - Ajustando Datas

###A) Colocando na Última Data Disponível

In [14]:
cvm_2021.sort_values(by = ['CNPJ_FUNDO','DT_COMPTC'], ascending = True, inplace = True)

In [15]:
cvm_2020.sort_values(by = ['CNPJ_FUNDO','DT_COMPTC'], ascending = True, inplace = True)

###B) Criando uma Lista de Cada CNPJ Disponível

In [16]:
cnpjs_2021 = list(cvm_2021['CNPJ_FUNDO'].unique())

In [17]:
cnpjs_2020 = list(cvm_2020['CNPJ_FUNDO'].unique())

###C) Criando uma Lista das Últimas Datas de Cada CNPJ Disponível

In [18]:
ultimas_datas_2021 = []

for cnpj in cnpjs_2021:
  ultimas_datas_2021.append(cvm_2021[cvm_2021['CNPJ_FUNDO'] == cnpj]['DT_COMPTC'].max())

In [19]:
ultimas_datas_2020 = []

for cnpj in cnpjs_2020:
  ultimas_datas_2020.append(cvm_2020[cvm_2020['CNPJ_FUNDO'] == cnpj]['DT_COMPTC'].max())

###D) Criando um DataFrame

In [20]:
df_ultimas_datas_2021 = pd.DataFrame({'CNPJ_FUNDO': cnpjs_2021, 'DT_COMPTC_ULTIMA': ultimas_datas_2021})

In [21]:
df_ultimas_datas_2020 = pd.DataFrame({'CNPJ_FUNDO': cnpjs_2020, 'DT_COMPTC_ULTIMA': ultimas_datas_2020})

###E) Mesclando com a Novo Coluna

In [22]:
df_final_2021 = pd.merge(cvm_2021, df_ultimas_datas_2021, on = 'CNPJ_FUNDO', how = 'left')

In [23]:
df_final_2020 = pd.merge(cvm_2020, df_ultimas_datas_2020, on = 'CNPJ_FUNDO', how = 'left')

###F) Deixando Apenas as Últimas Datas Disponíveis

In [24]:
deletar = []

for i in range(df_final_2021.shape[0]):
  if (df_final_2021.loc[i, 'DT_COMPTC'] < df_final_2021.loc[i, 'DT_COMPTC_ULTIMA']):
    deletar.append(i)

In [25]:
df_final_2021.drop(deletar, axis=0, inplace = True)
df_final_2021.index = range(df_final_2021.shape[0])

In [26]:
df_final_2021.drop_duplicates(subset = ['CNPJ_FUNDO', 'DT_COMPTC'], keep = 'first', inplace = True)
df_final_2021.index = range(df_final_2021.shape[0])

In [27]:
deletar = []

for i in range(df_final_2020.shape[0]):
  if (df_final_2020.loc[i, 'DT_COMPTC'] < df_final_2020.loc[i, 'DT_COMPTC_ULTIMA']):
    deletar.append(i)

In [28]:
df_final_2020.drop(deletar, axis=0, inplace = True)
df_final_2020.index = range(df_final_2020.shape[0])

In [29]:
df_final_2020.drop_duplicates(subset = ['CNPJ_FUNDO', 'DT_COMPTC'], keep = 'first', inplace = True)
df_final_2020.index = range(df_final_2020.shape[0])

##3.0 - Concatenando as Bases Anuais

In [30]:
df_final = pd.concat([df_final_2021, df_final_2020])

In [31]:
df_final = df_final[['CNPJ_FUNDO', 'DENOM_SOCIAL', 'DT_COMPTC', 'VL_PATRIM_LIQ', 'PUBLICO_ALVO', 'QT_COTA_SUBSCR', 'VL_CAP_SUBSCR', 'NR_TOTAL_COTST_SUBSCR', 'PR_TOTAL_COTA_SUBSCR']]

###A) Colocando na Última Data Disponível

In [32]:
df_final.sort_values(by = ['CNPJ_FUNDO','DT_COMPTC'], ascending = True, inplace = True)

###B) Criando uma Lista de Cada CNPJ Disponível

In [33]:
cnpjs_df_final = list(cvm_2021['CNPJ_FUNDO'].unique())

###C) Criando uma Lista das Últimas Datas de Cada CNPJ Disponível

In [34]:
ultimas_datas = []

for cnpj in cnpjs_df_final:
  ultimas_datas.append(df_final[df_final['CNPJ_FUNDO'] == cnpj]['DT_COMPTC'].max())

###D) Criando um DataFrame

In [35]:
df_ultimas_datas = pd.DataFrame({'CNPJ_FUNDO': cnpjs_df_final, 'DT_COMPTC_ULTIMA': ultimas_datas})

###E) Mesclando com a Novo Coluna

In [36]:
df_final_total = pd.merge(df_final, df_ultimas_datas, on = 'CNPJ_FUNDO', how = 'left')

###F) Deixando Apenas as Últimas Datas Disponíveis

In [37]:
deletar = []

for i in range(df_final_total.shape[0]):
  if (df_final_total.loc[i, 'DT_COMPTC'] < df_final_total.loc[i, 'DT_COMPTC_ULTIMA']):
    deletar.append(i)

In [38]:
df_final_total.drop(deletar, axis=0, inplace = True)
df_final_total.index = range(df_final_total.shape[0])

In [39]:
df_final_total.drop_duplicates(subset = ['CNPJ_FUNDO', 'DT_COMPTC'], keep = 'first', inplace = True)
df_final_total.index = range(df_final_total.shape[0])

In [40]:
df_final.sort_values(by = ['CNPJ_FUNDO','DT_COMPTC'], ascending = True, inplace = True)

In [41]:
df_final_total.sort_values(by = ['DT_COMPTC'], ascending = True, inplace = True)

#5.0 - Salndo a Base em CSV

In [42]:
bndes_informes_trimestrais = df_final_total[['CNPJ_FUNDO', 'DENOM_SOCIAL', 'DT_COMPTC', 'VL_PATRIM_LIQ', 'PUBLICO_ALVO', 'QT_COTA_SUBSCR', 'VL_CAP_SUBSCR', 'NR_TOTAL_COTST_SUBSCR', 'PR_TOTAL_COTA_SUBSCR']]

In [43]:
bndes_informes_trimestrais.to_csv('bndes_informes_trimestrais.csv', sep = ';', encoding = "ISO-8859-1",index = False)

In [44]:
bndes_informes_trimestrais.to_json('bndes_informes_trimestrais.json')